# Affordability

Median Rent (R): Rent as a percentage of median household income.

Availability of Housing (H): Ratio of available rentals to the population size.

Number of Beds (B): The number of bedrooms, a key indicator of property size

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession

In [2]:
# load data
df = pd.read_parquet("../data/curated/features_domain.parquet") 

In [3]:
# Set display option to show all columns
pd.set_option('display.max_columns', None)

In [4]:
# Display the first 5 rows of the dataframe
df.head()

,url,price,address,property_type,latitude,longitude,Beds,Baths,Parking,bond,extracted_price,geometry,index_right,sa2_code,sa2_name,chg_flag,chg_lbl,sa3_code,sa3_name,sa4_code,sa4_name,gcc_code,gcc_name,ste_code,ste_name,aus_code,aus_name,areasqkm,loci_uri,geometry_proj,suburb,property_id,nearest_station_id,train_station_distance_km,cbd_distance_km,nearest_hospital_id,nearest_hospital_distance,mean_stores,total_stores,count_shopping_centres,SA2 code_x,Region,ERP_2021,ERP_2026,ERP_2031,ERP_2036,PNPD_2021,PNPD_2026,PNPD_2031,PNPD_2036,POPD_2021,POPD_2026,POPD_2031,POPD_2036,OPD_2021,OPD_2026,OPD_2031,OPD_2036,HHS_2021,HHS_2026,HHS_2031,HHS_2036,SPD_2021,SPD_2026,SPD_2031,SPD_2036,OCC_2021,OCC_2026,OCC_2031,OCC_2036,population_growth_2021_2026,population_growth_2026_2031,population_growth_2031_2036,total_population_growth,Facility Count,erp_2022_no.,erp_2023_no.,erp_change_no.,erp_change_percentage,pop_change_natural_increase,pop_change_net_internal_migration,pop_change_oversea_migration,area_km2,population_density_persons_per_km2_2023,property_sa2_name,nearest_parkres_distance,nearest_parkres_name,parkres_sa2_name,closest_school,distance_to_closest_school,closest_independent_school,distance_to_closest_independent_school,total_value_of_items,total_number_of_items,crime_frequency,SA2 code_y,SA2 name,Earners,Median age of earners,Sum,Median,Mean,P80/P20,P80/P50,P20/P50,P10/P50,Gini coefficient,Top 1%,Top 5%,Top 10%,Lowest Quartile,Second Quartile,Third Quartile,Highest Quartile
0,https://www.domain.com.au/10-allara-court-donv...,"$1,400.00","10 Allara Court, Donvale VIC 3111",Townhouse,-37.774273,145.181126,4.0,3.0,2.0,9125.0,1400.0,POINT (145.1811258 -37.77427300000001),310.0,211021261,Donvale - Park Orchards,0,No change,21102,Manningham - East,211,Melbourne - Outer East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (989259.4173701586 -4291898.086618988),Donvale,0,19900,5.6596,25.93487,H5063,1.920280,NaN,NaN,NaN,211021261,Donvale - Park Orchards,16410.0,16758.344317,17104.792858,17452.526326,268.029273,295.811538,329.732136,361.211596,16141.970727,16462.532778,16775.060722,17091.31473,5719.325815,5867.52,6059.52,6251.52,2.822356,2.805705,2.768381,2.733945,5913,6112.0,6312.0,6512.0,0.967246,0.96,0.96,0.96,2.122756,2.06732,2.032959,6.223035,NaN,16405,16511,106,0.6,-77,-59,242,20.8,793.7,Donvale - Park Orchards,3541.030,Mullum Mullum Park,Donvale - Park Orchards,1759,0.345483,1724,1.086111,NaN,NaN,NaN,211021261,Donvale - Park Orchards,10426,48,872950324,53881,83728,6.95,2.24,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4
1,https://www.domain.com.au/7-pine-ridge-donvale...,$750 per week,"7 Pine Ridge, Donvale VIC 3111",House,-37.791251,145.175649,4.0,2.0,0.0,3259.0,750.0,POINT (145.1756489 -37.7912513),310.0,211021261,Donvale - Park Orchards,0,No change,21102,Manningham - East,211,Melbourne - Outer East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (988607.8235432974 -4293742.529802319),Donvale,1,19899,4.6923,23.77997,H5063,0.384624,NaN,NaN,NaN,211021261,Donvale - Park Orchards,16410.0,16758.344317,17104.792858,17452.526326,268.029273,295.811538,329.732136,361.211596,16141.970727,16462.532778,16775.060722,17091.31473,5719.325815,5867.52,6059.52,6251.52,2.822356,2.805705,2.768381,2.733945,5913,6112.0,6312.0,6512.0,0.967246,0.96,0.96,0.96,2.122756,2.06732,2.032959,6.223035,NaN,16405,16511,106,0.6,-77,-59,242,20.8,793.7,Donvale - Park Orchards,2536.544,Mullum Mullum Park,Donvale - Park Orchards,5019,0.986444,1724,2.472361,NaN,NaN,NaN,211021261,Donvale - Park Orchards,10426,48,872950324,53881,83728,6.95,2.24,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4
2,https://www.domain.com.au/20-mulsanne-way-donv...,$1300 per week,"20 Mulsanne Way, Donvale VIC 3111",House,-37.797232,145.181264,5.0,2.0,2.0,5649.0,1300.0,POINT (145.1812636 -37.7972323),310.0,211021261,Donvale - Park Orchards,0,No change,21102,Man

In [5]:
# Define weights for each factor in the formula
R_weight = 0.4
H_weight = 0.3
B_weight = 0.3

In [6]:
# Group by both SA2 name and SA2 code and calculate aggregate metrics for each SA2 region
sa2_grouped = df.groupby(['sa2_name', 'sa2_code']).agg({
    'extracted_price': 'median',  # Median rent price in each SA2
    'Median': 'median',  # Median household income for each SA2
    'Beds': 'mean',  # Average number of beds for properties in each SA2
    'erp_2023_no.': 'sum',  # Total population in each SA2
    'sa2_name': 'size'  # Number of properties in each SA2
}).rename(columns={'sa2_name': 'property_count', 'Median': 'Median_Income', 'erp_2023_no.': 'Population'}).reset_index()



In [7]:
# change data type of sa2_code to float64
sa2_grouped['sa2_code'] = sa2_grouped['sa2_code'].astype('float64')

In [8]:
# Calculate Median Rent (R) as rent as a percentage of median household income using 'extracted_price'
sa2_grouped['Median_Rent'] = sa2_grouped['extracted_price'] / sa2_grouped['Median_Income']
# availability of housing: Ratio of available rentals to the population in each SA2
sa2_grouped['availability'] = sa2_grouped['property_count'] / sa2_grouped['Population']

In [9]:
# show the first 5 rows
sa2_grouped.head(10)

,sa2_name,sa2_code,extracted_price,Median_Income,Beds,Population,property_count,Median_Rent,availability
0,Abbotsford,206071139.0,670.0,71394.0,1.884615,520416,52,0.009385,0.000100
1,Airport West,210011226.0,580.0,62051.0,2.857143,177744,21,0.009347,0.000118
2,Albert Park,206051128.0,650.0,73107.0,1.772727,758745,45,0.008891,0.000059
3,Alphington - Fairfield,206021110.0,660.0,68391.0,2.428571,199311,21,0.009650,0.000105
4,Altona,213021341.0,650.0,65149.0,2.960000,347475,25,0.009977,0.000072
5,Altona Meadows,213021342.0,520.0,56799.0,3.000000,298304,16,0.009155,0.000054
6,Altona North,213021343.0,665.0,61437.0,2.947368,584706,38,0.010824,0.000065
7,Ardeer - Albion,213011328.0,455.0,48307.0,2.291667,185064,24,0.009419,0.000130
8,Armadale,206061135.0,750.0,73321.0,2.309524,410298,42,0.010229,0.000102
9,Ascot Vale,206031113.0,680.0,67266.0,2.472222,566208,36,0.010109,0.000064


In [10]:
# ascending order of Median Rent
sa2_grouped.sort_values(by='Median_Rent', ascending=True).head(10)

,sa2_name,sa2_code,extracted_price,Median_Income,Beds,Population,property_count,Median_Rent,availability
87,East Melbourne,206041119.0,582.5,81648.0,1.500000,173400,34,0.007134,0.000196
131,Hurstbridge,209031210.0,435.0,57348.0,2.500000,7058,2,0.007585,0.000283
95,Essendon (West) - Aberfeldie,206031501.0,522.5,64802.0,2.312500,798144,48,0.008063,0.000060
91,Elwood,206051129.0,605.0,74437.0,2.088235,1051212,68,0.008128,0.000065
121,Hawthorn - South,207011520.0,550.0,67529.0,1.754386,775260,60,0.008145,0.000077
275,Yarraville,213031352.0,622.5,76253.0,2.642857,897120,56,0.008164,0.000062
153,Manor Lakes - Quandong,213051579.0,500.0,60503.0,3.811594,828138,69,0.008264,0.000083
194,Prahran - Windsor,206061136.0,600.0,72577.0,1.832061,2736180,135,0.008267,0.000049
41,Burwood (Vic.),207031165.0,347.5,41496.0,1.773585,756054,54,0.008374,0.000071
221,South Yarra - West,206041125.0,610.0,72567.0,1.972973,271560,40,0.008406,0.000147


In [11]:
# remove rows with infinite values
sa2_grouped = sa2_grouped.replace([np.inf, -np.inf], np.nan).dropna()

In [12]:
# normalize the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
sa2_grouped[['Median_Rent', 'Median_Income', 'Beds', 'Population', 'property_count', 'availability']] = scaler.fit_transform(sa2_grouped[['Median_Rent', 'Median_Income', 'Beds', 'Population', 'property_count', 'availability']])

In [13]:
# now get the score for each SA2 region
sa2_grouped['score'] = R_weight * sa2_grouped['Median_Rent'] - B_weight *  sa2_grouped['Beds'] + H_weight * sa2_grouped['availability']

In [14]:
# sort the dataframe by score in descending order
sa2_grouped = sa2_grouped.sort_values(by='score', ascending=False)
# show the first 5 rows
sa2_grouped.head(10)

,sa2_name,sa2_code,extracted_price,Median_Income,Beds,Population,property_count,Median_Rent,availability,score
103,Flemington Racecourse,206041120.0,700.0,0.512179,0.142857,0.000000,0.005277,0.256560,1.000000,0.359767
157,Melbourne CBD - North,206041504.0,630.0,0.000000,0.056122,1.000000,1.000000,0.694445,0.001180,0.261295
158,Melbourne CBD - West,206041505.0,627.5,0.194108,0.032011,0.757566,0.815303,0.422576,0.001459,0.159865
57,Clayton (North) - Notting Hill,212051567.0,530.0,0.060212,0.100840,0.105795,0.176781,0.444587,0.003686,0.148688
156,Melbourne CBD - East,206041503.0,597.5,0.222844,0.054250,0.248288,0.430079,0.359142,0.003856,0.128539
46,Carlton,206041117.0,550.0,0.209751,0.048666,0.498189,0.501319,0.314513,0.001213,0.111569
261,Warrandyte - Wonga Park,211021262.0,1700.0,0.600137,1.000000,0.001209,0.000000,1.000000,0.005267,0.101580
25,Box Hill,207031163.0,630.0,0.239514,0.182028,0.382586,0.324538,0.381722,0.000647,0.098275
222,Southbank (West) - South Wharf,206041508.0,710.0,0.582001,0.044335,0.104975,0.303430,0.227822,0.008095,0.080257
258,Wandin - Seville,211051285.0,734.0,0.491265,0.142857,0.001921,0.002639,0.298147,0.007342,0.078604


In [15]:
# save the dataframe to a parquet file
sa2_grouped.to_csv('../data/curated/affordability_scored.csv', index=False)